# Trajectory prediction

## Setup

In [ ]:
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from src.data.download import download_ais_data, download_file
from src.data.cleaning import process_multiple_zip_files
from src.data.preprocessing import (
    load_and_prepare_data,
    create_sequences,
    split_by_vessel,
    normalize_data,
)

from src.models import TrajectoryDataset, EncoderDecoderGRU, EncoderDecoderGRUWithAttention
from src.utils.model_utils import HaversineLoss, train_model, evaluate_model, create_prediction_sequences, predict_trajectories, plot_training_history, load_model_and_config, visualize_predictions
from src.visualization import plot_trajectory_comparison, create_prediction_map, create_trajectory_map
from src.utils import set_seed, haversine_distance
from src.utils import pointwise_haversine, mean_haversine_error, rmse_haversine, fde, dtw_distance_trajectory, dtw_batch_mean


sns.set_style("darkgrid")

# Constants
DATA_DIR = Path("data")
MODELS = [[EncoderDecoderGRU, "best_model_encoder_decoder.pt"], [EncoderDecoderGRUWithAttention, "best_model_encoder_decoder_with_attention.pt"]]
INPUT_HOURS = 2
OUTPUT_HOURS = 1
SAMPLING_RATE = 5
HIDDEN_SIZE = 256
NUM_LAYERS = 3
BATCH_SIZE = 512
EPOCHS = 100
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

set_seed(SEED)

print(u"Using device: ", DEVICE)

## Data

### Acquisition

In [ ]:
ZIP_NAMES = [
    "aisdk-2024-03-01.zip",
    # "aisdk-2024-03-02.zip",
    # "aisdk-2024-03-03.zip",
    # "aisdk-2024-03-04.zip",
    # "aisdk-2024-03-05.zip",
    # "aisdk-2024-03-06.zip",
    # "aisdk-2024-03-07.zip",
    # "aisdk-2024-03-08.zip",
    # "aisdk-2024-03-09.zip",
    # "aisdk-2024-03-10.zip"
]
# ZIP_NAMES = [] // Uncomment to download all files

if len(ZIP_NAMES) == 0:
    YEAR = "2024"
    MAX_WORKERS = 8
    download_ais_data(YEAR, DATA_DIR, MAX_WORKERS)
else:
    for ZIP_NAME in ZIP_NAMES:
        download_file("http://aisdata.ais.dk/2024/" + ZIP_NAME, DATA_DIR / ZIP_NAME)

### Cleaning

In [ ]:
process_multiple_zip_files(DATA_DIR)

In [ ]:
df = load_and_prepare_data(DATA_DIR)
sequences, targets, mmsi_labels, feature_cols = create_sequences(
    df, INPUT_HOURS, OUTPUT_HOURS, SAMPLING_RATE
)

X_train, X_val, X_test, y_train, y_val, y_test = split_by_vessel(
    sequences, targets, mmsi_labels, train_ratio=0.7, val_ratio=0.15, random_seed=42
)

X_train, X_val, X_test, y_train, y_val, y_test, input_scaler, output_scaler = normalize_data(
    X_train, X_val, X_test, y_train, y_val, y_test
)

train_dataset = TrajectoryDataset(X_train, y_train)
val_dataset = TrajectoryDataset(X_val, y_val)
test_dataset = TrajectoryDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

input_size = len(feature_cols)
output_timesteps = y_train.shape[1] // 2

### Overview

In [ ]:
print(f"Total rows: {len(df):,}")
print(f"Unique vessels (MMSI): {df['MMSI'].n_unique()}")
df.head()

In [ ]:
MAX_VESSELS = 100
create_trajectory_map(df, MAX_VESSELS)

## Training

In [ ]:
models = [ModelClass(
    input_size=input_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    output_seq_len=output_timesteps,
    dropout=0.3,
).to(DEVICE) for ModelClass, _ in MODELS]

print("\nModels architecture:")
for i, m in enumerate(models):
    print(f"\nModel #{i}:")
    print(m)
    print(f"Number of parameters: {sum(p.numel() for p in m.parameters()):,}")

total_params = sum(
    p.numel()
    for model in models
    for p in model.parameters()
)

print(f"\nTotal parameters: {total_params:,}")

In [ ]:
# params = [p for model in models for p in model.parameters()]

criterion = HaversineLoss(output_scaler).to(DEVICE)
# optimizer = torch.optim.AdamW(params, lr=LEARNING_RATE, weight_decay=1e-5)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=10)

In [ ]:
print(f"\nStarting training for {EPOCHS} epochs...")
train_losses = []
val_losses = []
best_val_loss = float("inf")
patience_counter = 0
early_stop_patience = 20

In [ ]:
def train_model_loop(model, model_path):    
    # reset things for this model
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=10
    )

    best_val_loss = float("inf")
    patience_counter = 0
    train_losses = []
    val_losses = []

    print(f"\nTraining model: {type(model).__name__}")


    for epoch in range(EPOCHS):
        teacher_forcing_ratio = max(0.2, 1.0 - (0.8 * (epoch / EPOCHS)))
        train_loss = train_model(model, train_loader, criterion, optimizer, DEVICE, epoch, EPOCHS, teacher_forcing_ratio)
        val_loss = evaluate_model(model, val_loader, criterion, output_scaler, DEVICE)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        scheduler.step(val_loss)

        print(f"Epoch [{epoch+1}/{EPOCHS}] - " f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": val_loss,
                    "input_scaler": input_scaler,
                    "output_scaler": output_scaler,
                    "config": {
                        "input_size": input_size,
                        "hidden_size": HIDDEN_SIZE,
                        "num_layers": NUM_LAYERS,
                        "output_seq_len": output_timesteps,
                        "input_hours": INPUT_HOURS,
                        "output_hours": OUTPUT_HOURS,
                        "sampling_rate": SAMPLING_RATE,
                        "feature_cols": feature_cols,
                    },
                },
                model_path,
            )
            print(f"  -> Saved best model (val_loss: {val_loss:.6f})")
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break

    print(f"\nFinished training model: {type(model).__name__}")
    print(f"\nTraining complete! Best validation loss: {best_val_loss:.6f}")

    plot_training_history(train_losses, val_losses)

for model, (_, model_path) in zip(models, MODELS):
    train_model_loop(model, model_path)

## Testing

In [ ]:
def test_model_loop(model, model_path):   
    checkpoint = torch.load(model_path, map_location=torch.device(DEVICE), weights_only=False)
    model.load_state_dict(checkpoint["model_state_dict"])
    test_loss, test_predictions, true_targets = evaluate_model(model, test_loader, criterion, output_scaler, DEVICE, return_predictions=True)
    print(f"Final Test Loss for {models}: {test_loss:.6f}")
    return test_predictions, true_targets
    
test_predictions = []
true_targets = []

for model, (_, model_path) in zip(models, MODELS):
    tp, tt = test_model_loop(model, model_path)
    test_predictions.append(tp)
    true_targets.append(tt)

test_predictions = np.asarray(test_predictions)
true_targets = np.asarray(true_targets)

In [ ]:
for model in models:
    visualize_predictions(model, test_loader, output_scaler, DEVICE)

## Results

In [ ]:
N_VESSELS = 25

def final_predictions(ModelClass, model_path):
    model, config, input_scaler, output_scaler = load_model_and_config(
        model_path, ModelClass, DEVICE
    )

    sequences, targets, mmsi_list, full_trajectories, timestamps_list = create_prediction_sequences(
        df, config, n_vessels=N_VESSELS
    )

    predictions, _ = predict_trajectories(model, sequences, input_scaler, output_scaler)

    # plot_trajectory_comparison(
    #     full_trajectories, 
    #     predictions, 
    #     mmsi_list, 
    #     config["output_hours"], 
    # )

    create_prediction_map(
        full_trajectories, 
        predictions, 
        mmsi_list, 
        config["output_hours"], 
        "output.html"
    )
    
for ModelClass, model_path in MODELS:
    final_predictions(ModelClass, model_path)

# Statistics

In [ ]:
for model, tp, tt in zip(models, test_predictions, true_targets):
    print(f"Model: {type(model).__name__}")
    
    # Ground truth and predictions, both (N, 2*T)
    y_pred = tp  # predicted trajectories
    y_true = tt  # ground truth

    # ---- reshape to (N, T, 2) generically ----
    N, D = y_true.shape          # N = number of samples, D = 2 * T
    T = D // 2                   # T = number of timesteps

    y_true = y_true.reshape(N, T, 2)
    y_pred = y_pred.reshape(N, T, 2)

    print("y_true shape:", y_true.shape)  
    print("y_pred shape:", y_pred.shape) 
    print()
    # ---- now metrics will work ----
    print("Mean Haversine Error (km):", mean_haversine_error(y_true, y_pred))
    print("Root Mean Squared Error (km):", rmse_haversine(y_true, y_pred))
    print("Final Displacement (km):", fde(y_true, y_pred))
    print("Dynamic Time Warping Distance (km):", dtw_batch_mean(y_true, y_pred))
    print("\n\n")

